In [1]:
import csv
import folium
from folium.plugins import HeatMap, HeatMapWithTime
from IPython.display import display, clear_output, HTML 
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import time
import pandas as pd
import os
import ipywidgets as widgets
import imageio
from PIL import Image
from selenium import webdriver



In [2]:
# pre-process the delitos dataset
def sort_csv_file(filename):
    base_name, ext = os.path.splitext(filename)
    output_filename = f'{base_name}_sorted{ext}'

    with open(filename, 'r', newline='') as file:
        reader = csv.DictReader(file)
        sorted_rows = sorted(reader, key=lambda row: (row['fecha'], row['hora']))

    # Write the sorted rows to a new CSV file with the modified filename
    with open(output_filename, 'w', newline='') as output_file:
        fieldnames = reader.fieldnames
        writer = csv.DictWriter(output_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(sorted_rows)

In [3]:
def plot_crime_heatmap(csv_file):
    # Read the sorted CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Create a map centered on a specific location
    latitude, longitude = df['y'].mean(), df['x'].mean()
    crime_map = folium.Map(location=[latitude, longitude], zoom_start=12)

    # Create a list of latitudes and longitudes for the heat map data
    heat_data = [[row['y'], row['x']] for _, row in df.iterrows()]

    # Add the heat map layer to the map with adjusted radius and blur
    HeatMap(heat_data, radius=10, blur=5).add_to(crime_map)

     # Display the map in a separate window
    display(crime_map)

In [4]:
filename = 'data_homicides_sorted.csv'  # Replace with the actual filename of the sorted CSV file
plot_crime_heatmap(filename)

DYANMIC HEAT MAP! 

In [5]:
def create_base_map(csv_file):
    # Read the sorted CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file)

    # Convert 'fecha' column to datetime type
    df['fecha'] = pd.to_datetime(df['fecha'])

    # Group the data into 7-day increments (weekly)
    df['week'] = df['fecha'].dt.to_period('W').dt.start_time

    # Get unique weeks from the DataFrame
    unique_weeks = df['week'].unique()

    # Create a map centered on a specific location
    latitude, longitude = df['y'].mean(), df['x'].mean()

    # Create the base map
    crime_map = folium.Map(location=[latitude, longitude], zoom_start=12)

    # Create an empty HeatMap layer
    heatmap_layer = HeatMap([], radius=10, blur=5)

    # Add the HeatMap layer to the map
    heatmap_layer.add_to(crime_map)

    return crime_map, heatmap_layer, df, unique_weeks

def update_heatmap(crime_map, heatmap_layer, df, week):
    # Filter the DataFrame for the selected week
    filtered_df = df[df['week'] == week]

    # Create a list of latitudes and longitudes for the heat map data for the selected week
    heat_data = [[row['y'], row['x']] for _, row in filtered_df.iterrows()]

    # Update the data in the HeatMap layer
    heatmap_layer.data = heat_data

    # Clear previous output and display the updated map without flashing white
    display(crime_map)
    clear_output(wait=True)

def plot_weekly_heatmap(crime_map, heatmap_layer, df, unique_weeks):
    # Iterate over each week to create the animation-like effect
    for week in sorted(unique_weeks):
        update_heatmap(crime_map, heatmap_layer, df, week)
        time.sleep(1)  # Adjust the sleep duration (in seconds) to control the animation speed

# Example usage
filename = 'data_homicides_sorted.csv'  # Replace with the actual filename of the sorted CSV file
crime_map, heatmap_layer, df, unique_weeks = create_base_map(filename)
plot_weekly_heatmap(crime_map, heatmap_layer, df, unique_weeks)


KeyboardInterrupt: 

In [22]:
from selenium import webdriver

def update_heatmap(crime_map, heatmap_layer, df, week):
    # Filter the DataFrame for the selected week
    filtered_df = df[df['week'] == week]

    # Create a list of latitudes and longitudes for the heat map data for the selected week
    heat_data = [[row['y'], row['x']] for _, row in filtered_df.iterrows()]

    # Update the data in the HeatMap layer
    heatmap_layer.data = heat_data

def plot_weekly_heatmap(crime_map, heatmap_layer, df, unique_weeks):
    # Create a directory to save frames
    if not os.path.exists('frames'):
        os.makedirs('frames')
    else:
        for filename in os.listdir('frames'):
            os.remove(f'frames/{filename}')

    filenames = []

    # Set up a Selenium WebDriver for Chrome
    driver = webdriver.Chrome()

    # Iterate over each week to create the animation-like effect
    for i, week in enumerate(sorted(unique_weeks)):
        update_heatmap(crime_map, heatmap_layer, df, week)
        
        # Save the map as an HTML file
        html_file = f'map_{i}.html'
        crime_map.save(html_file)

        # Open the map in the browser and take a screenshot
        driver.get(f'file://{os.path.abspath(html_file)}')
        png_file = f'frames/frame_{i}.png'
        filenames.append(png_file)
        #Add a slight delay
        time.sleep(.13)
        driver.save_screenshot(png_file)
        
        # Now delete the HTML file to avoid cluttering up the directory
        os.remove(html_file)

    driver.quit()

    # Create a GIF animation from the saved frames
    imageio.mimsave('animation.gif', [imageio.imread(filename) for filename in filenames])

filename = 'data_homicides_2.csv'  # Replace with the actual filename of the sorted CSV file
crime_map, heatmap_layer, df, unique_weeks = create_base_map(filename)
plot_weekly_heatmap(crime_map, heatmap_layer, df, unique_weeks)

/var/folders/dt/0bnsl48d14n491d2_r6d8fvw0000gn/T/ipykernel_69322/2197084030.py:48: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imageio.mimsave('animation.gif', [imageio.imread(filename) for filename in filenames])
